## Data Location

In [0]:
%fs
ls /FileStore/tables/project_sales_analysis

path,name,size,modificationTime
dbfs:/FileStore/tables/project_sales_analysis/menu.txt,menu.txt,98,1750354459000
dbfs:/FileStore/tables/project_sales_analysis/sales.txt,sales.txt,3465,1750354468000


## Creating a Dataframe

In [0]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, DateType

schema = StructType([
    StructField("product_id", IntegerType(), True),
    StructField("customer_id", StringType(), True),
    StructField("order_date", DateType(), True),
    StructField("location", StringType(), True),
    StructField("source_order", StringType(), True)
])

sales_df = spark.read.format("csv").option("inferschema", "true").schema(schema).load("dbfs:/FileStore/tables/project_sales_analysis/sales.txt")
display(sales_df.limit(5))

product_id,customer_id,order_date,location,source_order
1,A,2023-01-01,India,Swiggy
2,A,2022-01-01,India,Swiggy
2,A,2023-01-07,India,Swiggy
3,A,2023-01-10,India,Restaurant
3,A,2022-01-11,India,Swiggy


In [0]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, DateType

schema = StructType([
    StructField("product_id", IntegerType(), True),
    StructField("product_name", StringType(), True),
    StructField("price", StringType(), True),
])

menu_df = spark.read.format("csv").option("inferschema", "true").schema(schema).load("dbfs:/FileStore/tables/project_sales_analysis/menu.txt")
display(menu_df.limit(5))

product_id,product_name,price
1,PIZZA,100
2,Chowmin,150
3,sandwich,120
4,Dosa,110
5,Biryani,80


### Add Year, Month, Quarter

In [0]:
from pyspark.sql.functions import year, quarter, month

sales_df = sales_df.withColumn("order_year", year(sales_df.order_date))
sales_df = sales_df.withColumn("order_quarter", quarter(sales_df.order_date))
sales_df = sales_df.withColumn("order_month", month(sales_df.order_date))

display(sales_df.limit(5))

product_id,customer_id,order_date,location,source_order,order_year,order_quarter,order_month
1,A,2023-01-01,India,Swiggy,2023,1,1
2,A,2022-01-01,India,Swiggy,2022,1,1
2,A,2023-01-07,India,Swiggy,2023,1,1
3,A,2023-01-10,India,Restaurant,2023,1,1
3,A,2022-01-11,India,Swiggy,2022,1,1


## Deriving KPI's

#### 1) Total amount spent by each customer

In [0]:
total_amount_spent = sales_df.join(menu_df, 'product_id').groupBy('customer_id').agg({'price':'sum'}).orderBy('customer_id')
display(total_amount_spent.limit(5))

customer_id,sum(price)
A,4260.0
B,4440.0
C,2400.0
D,1200.0
E,2040.0


Databricks visualization. Run in Databricks to view.

#### 2) Total Amount spent on each food category

In [0]:
total_amount_spent_on_category = sales_df.join(menu_df, 'product_id').groupBy('product_name').agg({'price':'sum'}).orderBy('product_name')
display(total_amount_spent_on_category.limit(5))

product_name,sum(price)
Biryani,480.0
Chowmin,3600.0
Dosa,1320.0
PIZZA,2100.0
Pasta,1080.0


Databricks visualization. Run in Databricks to view.

#### 3) Monthly sales

In [0]:
total_monthly_sales = sales_df.join(menu_df, 'product_id').groupBy('order_month').agg({'price':'sum'}).orderBy('order_month')
display(total_monthly_sales)

order_month,sum(price)
1,2960.0
2,2730.0
3,910.0
5,2960.0
6,2960.0
7,910.0
11,910.0


Databricks visualization. Run in Databricks to view.

#### 4) Quarterly sales

In [0]:
total_quarterly_sales = sales_df.join(menu_df, 'product_id').groupBy('order_quarter').agg({'price':'sum'}).orderBy('order_quarter')
display(total_quarterly_sales)

order_quarter,sum(price)
1,6600.0
2,5920.0
3,910.0
4,910.0


Databricks visualization. Run in Databricks to view.

#### 5) Yearly sales

In [0]:
total_yearly_sales = sales_df.join(menu_df, 'product_id').groupBy('order_year').agg({'price':'sum'}).orderBy('order_year')
display(total_yearly_sales)

order_year,sum(price)
2022,4350.0
2023,9990.0


Databricks visualization. Run in Databricks to view.

#### 6) Number of orders per category

In [0]:
from pyspark.sql.functions import count

orders_per_category = sales_df.join(menu_df, 'product_id') \
                .groupBy('product_id', 'product_name') \
                .agg(count('product_id').alias('product_count')) \
                .orderBy('product_count', ascending=0) \
                .drop('product_id')
display(orders_per_category)

product_name,product_count
sandwich,48
Chowmin,24
PIZZA,21
Dosa,12
Biryani,6
Pasta,6


Databricks visualization. Run in Databricks to view.

#### 7) Top ordered item

In [0]:
from pyspark.sql.functions import count

most_purchased = sales_df.join(menu_df, 'product_id') \
                .groupBy('product_id', 'product_name') \
                .agg(count('product_id').alias('product_count')) \
                .orderBy('product_count', ascending=0) \
                .drop('product_id') \
                .limit(1)
display(most_purchased)

product_name,product_count
sandwich,48


Databricks visualization. Run in Databricks to view.

#### 8) Frequency of customers visited to restaurant

In [0]:
from pyspark.sql.functions import countDistinct

visited_restaurant = sales_df.filter(sales_df.source_order == 'Restaurant') \
            .groupBy('customer_id') \
            .agg(countDistinct('order_date'))

display(visited_restaurant)

customer_id,count(order_date)
E,5
B,6
D,1
C,3
A,6


Databricks visualization. Run in Databricks to view.

#### 9) Total sales by each country

In [0]:
sales_by_country = sales_df.join(menu_df, 'product_id').groupBy('location').agg({'price':'sum'})
display(sales_by_country.limit(5))

location,sum(price)
India,4860.0
USA,2460.0
UK,7020.0


Databricks visualization. Run in Databricks to view.

#### 10) Total sales by order_source

In [0]:
sales_by_source = sales_df.join(menu_df, 'product_id').groupBy('source_order').agg({'price':'sum'})
display(sales_by_source.limit(5))

source_order,sum(price)
zomato,4920.0
Swiggy,6330.0
Restaurant,3090.0


Databricks visualization. Run in Databricks to view.